<font size="+3.8">Scanpy single-cell pre-processing</font>  
<font size="+1.5"></font>  

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
import os
from datetime import date
from matplotlib.pyplot import rc_context

In [ ]:
import utils

In [ ]:
sc.settings.verbosity = 3

In [ ]:
main_dir='/run/user/1000/gvfs/smb-share:server=138.245.4.35,share=bd-dichgans/SF' # Linux

In [ ]:
main_dir='\\\isdsynnas.srv.med.uni-muenchen.de\BD-Dichgans\SF' # Win

# Load data

Downloaded from: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE133283 06/03/2023

Note: Data provided unprocessed and not annotated per animal.

In [ ]:
t = pd.read_csv(os.path.join(main_dir, "P6_vasc_scRNAseq", "Jeong2022", "count_matrix", "GSM3904816_Adult-1_gene_counts.tsv.gz"), sep="\t",index_col=None)
t["count"].value_counts()

In [ ]:
t

In [ ]:
t=pd.read_csv(os.path.join(main_dir, "P6_vasc_scRNAseq", "Jeong2022", "count_matrix", "GSM3904817_Adult-2_gene_counts.tsv.gz"), sep="\t")
t = t.pivot(index='cell', columns='gene', values='count')
t = t.fillna(0)
adata = sc.AnnData(t)
adata

In [ ]:
t

In [ ]:
import glob
path = os.path.join(main_dir, 'P6_vasc_scRNAseq', 'Jeong2022', 'count_matrix')
all_files = glob.glob(os.path.join(path , "*.tsv.gz"))
all_files

In [ ]:
all_files[0]

In [ ]:
index = all_files[0].find("GSM")
index

In [ ]:
gsm_number = (all_files[0].split('_')[0]
              .split('GSM')[1])
print(gsm_number)

In [ ]:
li = []

for filename in all_files:
    df = pd.read_csv(filename, sep="\t",index_col=None)
    df["cell"] = "GSM"+filename.split('GSM')[1].split('_')[0]+"_"+filename.split('GSM')[1].split('_')[1]+"_"+df["cell"] # add gsm and sample to cell id
    #df["cell"] = df["cell"]+
    df = df.pivot(index='cell', columns='gene', values='count')
    df = df.fillna(0)
    adata = sc.AnnData(df)   
    li.append(df)
#all = pd.concat(li, axis=1, ignore_index=True)

In [ ]:
li

In [ ]:
all = pd.concat(li, axis=1)

In [ ]:
all

In [ ]:
all.pivot(index='cell', columns='gene', values='count')


In [ ]:
ad = sc.read_text(main_dir+"\\P6_vasc_scRNAseq\Jeong2022\\GSM3904817_Adult-2_gene_counts.tsv.gz")
ad

In [ ]:
ad.obs.head()

In [ ]:
ad.var_names

In [ ]:
# to be continued..